In [1]:
import os
import pandas as pd
from sqlalchemy import create_engine

# путь к БД на вашем компьютере (например, в той же папке, что и тетрадь)
path_to_db_local = '../datasets/telecomm_csi.db'
# путь к БД на платформе
path_to_db_platform = '/datasets/telecomm_csi.db'
# итоговый путь к БД
path_to_db = None

# если путь на вашем компьютере ведёт к БД, то он становится итоговым
if os.path.exists(path_to_db_local):
    path_to_db = path_to_db_local
# иначе: если путь на платформе ведёт к БД, то он становится итоговым
elif os.path.exists(path_to_db_platform):
    path_to_db = path_to_db_platform
# иначе выводится сообщение о том, что файл не найден
else:
    raise Exception('Файл с базой данных SQLite не найден!')

# если итоговый путь не пустой
if path_to_db:
    # то создаём подключение к базе
    engine = create_engine(f'sqlite:///{path_to_db}', echo=False)

    # пример запроса
    query = """
            SELECT u.user_id,
                      u.lt_day,
                      CASE 
                        WHEN u.lt_day <= 365 THEN 1
                        ELSE 0
                      END as is_new,
                      u.age,
                      CASE 
                        WHEN u.gender_segment = 1 THEN 'женщина'
                        WHEN u.gender_segment = 0 THEN 'мужчина'
                        ELSE 'undefined'
                      END as gender_segment,
                      u.os_name,
                      u.cpe_type_name,
                      l.country,
                      l.city,
                      SUBSTR(ageseg.title, 3) as age_segment,
                      SUBSTR(ts.title, 3) as traffic_segment,
                      SUBSTR(ls.title, 3) as lifetime_segment,
                      u.nps_score,
                      CASE 
                        WHEN u.nps_score >= 9 THEN 'cторонники'
                        WHEN u.nps_score <= 6 THEN 'критики'
                        ELSE 'нейтралы'
                      END as nps_group
                FROM user u
                LEFT JOIN location l ON u.location_id = l.location_id
                LEFT JOIN age_segment ageseg ON u.age_gr_id = ageseg.age_gr_id
                LEFT JOIN traffic_segment ts ON u.tr_gr_id = ts.tr_gr_id
                LEFT JOIN lifetime_segment ls ON u.lt_gr_id = ls.lt_gr_id;
            """

    # создаём датафрейм по данным запроса
    df = pd.read_sql(query, engine)

In [2]:
df.head()

,user_id,lt_day,is_new,age,3,gender_segment,os_name,cpe_type_name,country,city,age_segment,traffic_segment,lifetime_segment,nps_score,nps_group
0,A001A2,2320,0,45.0,3,женщина,ANDROID,SMARTPHONE,Россия,Уфа,45-54,1-5,36+,10,cторонники
1,A001WF,2344,0,53.0,3,мужчина,ANDROID,SMARTPHONE,Россия,Киров,45-54,1-5,36+,10,cторонники
2,A003Q7,467,0,57.0,3,мужчина,ANDROID,SMARTPHONE,Россия,Москва,55-64,20-25,13-24,10,cторонники
3,A004TB,4190,0,44.0,3,женщина,IOS,SMARTPHONE,Россия,РостовнаДону,35-44,0.1-1,36+,10,cторонники
4,A004XT,1163,0,24.0,3,мужчина,ANDROID,SMARTPHONE,Россия,Рязань,16-24,5-10,36+,10,cторонники


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 502493 entries, 0 to 502492
Data columns (total 15 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   user_id           502493 non-null  object 
 1   lt_day            502493 non-null  int64  
 2   is_new            502493 non-null  int64  
 3   age               501939 non-null  float64
 4   3                 502493 non-null  int64  
 5   gender_segment    502493 non-null  object 
 6   os_name           502493 non-null  object 
 7   cpe_type_name     502493 non-null  object 
 8   country           502493 non-null  object 
 9   city              502493 non-null  object 
 10  age_segment       502493 non-null  object 
 11  traffic_segment   502493 non-null  object 
 12  lifetime_segment  502493 non-null  object 
 13  nps_score         502493 non-null  int64  
 14  nps_group         502493 non-null  object 
dtypes: float64(1), int64(4), object(10)
memory usage: 57.5+ MB


In [4]:
df.to_csv('nps.csv')

https://public.tableau.com/views/nps_16877774574330/NPS_2